<a href="https://colab.research.google.com/github/sunny0103/DeepLearning_nlp_projects/blob/main/meeting_summarization/meeting_transcript_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[torch] datasets evaluate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.6 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=56c1e3b91f1c2dc6ccb603ff61aa8831ad94dc90e56938ea

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
cd "/content/drive/MyDrive/Data/meeting_transcript"

/content/drive/MyDrive/Data/meeting_transcript


In [ ]:
import random, os

import pandas as pd
import numpy as np
import json

import torch
from transformers import (AutoTokenizer,
                          DataCollatorForSeq2Seq,
                          AutoModelForSeq2SeqLM,
                          Seq2SeqTrainingArguments,
                          Seq2SeqTrainer)
import evaluate

from datasets import load_dataset

In [ ]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything(42)

In [ ]:
# TRAIN_SOURCE = os.path.join("./train.json")
# TEST_SOURCE = os.path.join("./test.json")

In [ ]:
# with open(TRAIN_SOURCE) as f:
#     TRAIN_DATA = json.loads(f.read())

# with open(TEST_SOURCE) as f:
#     TEST_DATA = json.loads(f.read())

In [ ]:
# TRAIN_DATA[0]

In [ ]:
# train = pd.DataFrame(columns=['uid', 'title', 'region', 'context', 'summary'])
# uid = 1000
# for data in TRAIN_DATA:
#     for agenda in data['context'].keys():
#         context = ''
#         for line in data['context'][agenda]:
#             context += data['context'][agenda][line]
#             context += ' '
#         train.loc[uid, 'uid'] = uid
#         train.loc[uid, 'title'] = data['title']
#         train.loc[uid, 'region'] = data['region']
#         train.loc[uid, 'context'] = context[:-1]
#         train.loc[uid, 'summary'] = data['label'][agenda]['summary']
#         uid += 1


# test = pd.DataFrame(columns=['uid', 'title', 'region', 'context'])
# uid = 2000
# for data in TEST_DATA:
#     for agenda in data['context'].keys():
#         context = ''
#         for line in data['context'][agenda]:
#             context += data['context'][agenda][line]
#             context += ' '
#         test.loc[uid, 'uid'] = uid
#         test.loc[uid, 'title'] = data['title']
#         test.loc[uid, 'region'] = data['region']
#         test.loc[uid, 'context'] = context[:-1]
#         uid += 1

In [ ]:
# train['text'] = train.title + ' ' + train.region + ' ' + train.context
# test['text'] = test.title + ' ' + test.region + ' ' + test.context

In [ ]:
# train = train.drop(['title','region','context'], axis=1)
# train.head()

In [ ]:
# test = test.drop(['title','region','context'], axis=1)
# test.head()

In [ ]:
# train.to_csv('./train.csv', index=False)
# test.to_csv('./test.csv', index=False)

In [ ]:
raw_dataset = load_dataset("csv", data_files='./train.csv',split='train')
dataset = raw_dataset.train_test_split(.2)
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['uid', 'summary', 'text'],
        num_rows: 2395
    })
    test: Dataset({
        features: ['uid', 'summary', 'text'],
        num_rows: 599
    })
})

In [ ]:
dataset['train'][0]

{'uid': 3200,
 'summary': '제113회 임시회 회의록 서명의원으로 남궁유 의원, 김천봉 의원이 선출됨.',
 'text': '제113회    본회의 제1차(2002.02.06.) 음성 의사일정 제2항, 회의록서명의원선출의건을 상정합니다. 제113회 임시회 회의록 서명의원으로는 여러 의원님들이 사전에 양해하여 주신대로 남궁유 의원님과 김천봉 의원님을 선출하고자 하는데 의원 여러분, 이의가 없으십니까? (「없습니다」하는 의원 있음) 이의가 없으므로 남궁유 의원님과 김천봉 의원님이 회의록 서명의원으로 선출되었음을 선포합니다. 두 분 의원님께서는 수고하여 주시기 바랍니다.'}

In [ ]:
checkpoint = "psyche/KoT5-summarization"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/2395 [00:00<?, ? examples/s]

Map:   0%|          | 0/599 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
rouge = evaluate.load("rouge")

In [ ]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True )
  prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  result['gen_len'] = np.mean(prediction_lens)

  return {k: round(v, 4) for k, v in result.items()}


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
LEARNING_RATE = 2e-5
BATCH_SIZE = 16
WEIGHT_DECAY = 0.01
EPOCHS = 1

In [ ]:

training_args = Seq2SeqTrainingArguments(
    output_dir = 'base_model',
    evaluation_strategy ='epoch',
    learning_rate = LEARNING_RATE,
    per_device_train_batch_size= BATCH_SIZE,
    per_device_eval_batch_size = BATCH_SIZE,
    weight_decay = WEIGHT_DECAY,
    num_train_epochs = EPOCHS,
    predict_with_generate = True,
    # fp16=True
)

trainer  = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer = tokenizer,
    data_collator=data_collator,
    compute_metrics = compute_metrics
)

trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
